<a href="https://colab.research.google.com/github/Iqra-01/Google_AI_studio_Genai/blob/main/templates/character%20designer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-pro' # @param {isTemplate: true}
contents_b64 = 'W3sicGFydHMiOlt7InRleHQiOiJXcml0ZSBhIGNoYXJhY3RlciBkZXNpZ24gaW4gYSBwaXJhdGUtdGhlbWVkIGdhbWUgc2V0IGluIHRoZSBKb3Nlb24gZXJhLiJ9LHsidGV4dCI6ImlucHV0OiBNYWluIENoYXJhY3RlciAoRmVtYWxlKSJ9LHsidGV4dCI6Im91dHB1dDogKipOYW1lOioqIEhhZW55ZW9cXG5cXG4qKkFwcGVhcmFuY2U6KipcXG5cXG4qICoqQWdlOioqIE1pZC10d2VudGllc1xcbiogKipIZWlnaHQ6KiogNSc3XFxcIlxcbiogKipXZWlnaHQ6KiogMTMwIGxic1xcbiogKipCdWlsZDoqKiBBdGhsZXRpYyBhbmQgYWdpbGVcXG4qICoqSGFpcjoqKiBMb25nLCBibGFjayBoYWlyIHdvcm4gaW4gYSBwcmFjdGljYWwgYnJhaWRcXG4qICoqRXllczoqKiBQaWVyY2luZyBoYXplbCBleWVzIHRoYXQgcmVmbGVjdCB0aGUgc2VhJ3MgZGVwdGhzXFxuKiAqKkNsb3RoaW5nOioqXFxuICAgICogTG9vc2UsIGRhcmstY29sb3JlZCBwYW50cyB0dWNrZWQgaW50byBzdHVyZHkgbGVhdGhlciBib290c1xcbiAgICAqIEEgbGlnaHR3ZWlnaHQgY290dG9uIHNoaXJ0IHdpdGggYSBsb3cgbmVja2xpbmUsIHRyaW1tZWQgd2l0aCBpbnRyaWNhdGUgZW1icm9pZGVyeVxcbiAgICAqIEEgbGVhdGhlciB2ZXN0IHdpdGggbXVsdGlwbGUgcG9ja2V0cyBmb3IgdG9vbHMgYW5kIHN1cHBsaWVzXFxuKiAqKkFjY2Vzc29yaWVzOioqXFxuICAgICogQSBicm9hZC1icmltbWVkIGhhdCB0byBzaGllbGQgaGVyIGZhY2UgZnJvbSB0aGUgc3VuXFxuICAgICogQSBwYWlyIG9mIHNpbHZlciBlYXJyaW5ncyBpbiB0aGUgc2hhcGUgb2YgYW5jaG9yc1xcbiAgICAqIEEgbGVhdGhlciBiZWx0IHdpdGggYSBzbWFsbCBwb3VjaCBmb3IgdmFsdWFibGVzXFxuXFxuKipQZXJzb25hbGl0eToqKlxcblxcbiogKipEZXRlcm1pbmVkOioqIEhhZW55ZW8gaXMgYSBmZWFybGVzcyBhbmQgdGVuYWNpb3VzIHBpcmF0ZSwgZHJpdmVuIGJ5IGEgc3Ryb25nIHNlbnNlIG9mIGp1c3RpY2UgYW5kIGEgZGVzaXJlIGZvciBhZHZlbnR1cmUuXFxuKiAqKlJlc291cmNlZnVsOioqIEhlciB0aW1lIHNwZW50IGFzIGEgaGFlbnllbyAoS29yZWFuIGZlbWFsZSBkaXZlcikgaGFzIGhvbmVkIGhlciBzdXJ2aXZhbCBpbnN0aW5jdHMgYW5kIGFkYXB0YWJpbGl0eS5cXG4qICoqRW1wYXRoZXRpYzoqKiBEZXNwaXRlIGhlciBwaXJhdGUgbGlmZXN0eWxlLCBIYWVueWVvIGhhcyBhIGNvbXBhc3Npb25hdGUgaGVhcnQgYW5kIGEgc29mdCBzcG90IGZvciB0aG9zZSBpbiBuZWVkLlxcbiogKipJbnRlbGxpZ2VudDoqKiBTaGUgaXMgcXVpY2std2l0dGVkIGFuZCBvYnNlcnZhbnQsIGFibGUgdG8gYXNzZXNzIHNpdHVhdGlvbnMgYW5kIG1ha2UgZGVjaXNpb25zIHN3aWZ0bHkuXFxuKiAqKkluZGVwZW5kZW50OioqIEhhZW55ZW8gdmFsdWVzIGhlciBmcmVlZG9tIGFuZCByZWxpZXMgb24gaGVyIG93biBhYmlsaXRpZXMgcmF0aGVyIHRoYW4gc2Vla2luZyBoZWxwIGZyb20gb3RoZXJzLlxcblxcbioqU2tpbGxzOioqXFxuXFxuKiAqKkRpdmluZzoqKiBIYWVueWVvIGlzIGFuIGV4cGVydCBkaXZlciwgY2FwYWJsZSBvZiBob2xkaW5nIGhlciBicmVhdGggZm9yIGV4dGVuZGVkIHBlcmlvZHMgYW5kIG5hdmlnYXRpbmcgdW5kZXJ3YXRlciB3aXRoIGVhc2UuXFxuKiAqKlN3aW1taW5nOioqIEhlciBleGNlcHRpb25hbCBzd2ltbWluZyBhYmlsaXRpZXMgYWxsb3cgaGVyIHRvIG5hdmlnYXRlIHRoZSB0cmVhY2hlcm91cyBvY2VhbiBjdXJyZW50cyBhbmQgYXZvaWQgZGFuZ2VyLlxcbiogKipTcGVhcmZpc2hpbmc6KiogU2hlIGlzIGEgc2tpbGxlZCBzcGVhcmZpc2hlciwgdXNpbmcgaGVyIGtub3dsZWRnZSBvZiBtYXJpbmUgbGlmZSB0byBsb2NhdGUgYW5kIGNhdGNoIHByZXkgdW5kZXJ3YXRlci5cXG4qICoqU2VhbWFuc2hpcDoqKiBBcyBhIHBpcmF0ZSwgSGFlbnllbyBpcyBmYW1pbGlhciB3aXRoIHRoZSBiYXNpY3Mgb2Ygc2FpbGluZyBhbmQgc2hpcCBuYXZpZ2F0aW9uLlxcbiogKipIYW5kLXRvLUhhbmQgQ29tYmF0OioqIFdoaWxlIG5vdCBhbiBleHBlcnQgZmlnaHRlciwgc2hlIGlzIHByb2ZpY2llbnQgaW4gYmFzaWMgc2VsZi1kZWZlbnNlIHRlY2huaXF1ZXMsIHVzaW5nIGhlciBhZ2lsaXR5IGFuZCBrbm93bGVkZ2Ugb2YgcHJlc3N1cmUgcG9pbnRzIHRvIHN1YmR1ZSBvcHBvbmVudHMuXFxuXFxuKipCYWNrZ3JvdW5kOioqXFxuXFxuSGFlbnllbyB3YXMgcmFpc2VkIG9uIGEgY29hc3RhbCB2aWxsYWdlIGFuZCBsZWFybmVkIHRoZSB3YXlzIG9mIHRoZSBoYWVueWVvIGZyb20gYSB5b3VuZyBhZ2UuIEhvd2V2ZXIsIHdoZW4gaGVyIHZpbGxhZ2Ugd2FzIGF0dGFja2VkIGJ5IGJhbmRpdHMsIHNoZSB3aXRuZXNzZWQgZmlyc3RoYW5kIHRoZSBicnV0YWxpdHkgb2Ygd2FyIGFuZCBkZWNpZGVkIHRvIGZpZ2h0IGJhY2suXFxuXFxuSm9pbmluZyBhIHBpcmF0ZSBjcmV3LCBIYWVueWVvIGhhcyBwcm92ZW4gaGVyc2VsZiBhcyBhIHZhbHVhYmxlIG1lbWJlciwgdXNpbmcgaGVyIGRpdmluZyBza2lsbHMgdG8gc2NvdXQgZm9yIGVuZW15IHNoaXBzIGFuZCBnYXRoZXIgdW5kZXJ3YXRlciByZXNvdXJjZXMuIEhlciBkZXRlcm1pbmF0aW9uIGFuZCBjb21wYXNzaW9uIGhhdmUgYWxzbyBlYXJuZWQgaGVyIHRoZSByZXNwZWN0IG9mIGhlciBjcmV3bWF0ZXMuIn0seyJ0ZXh0IjoiaW5wdXQ6IE1haW4gQ2hhcmFjdGVyIChNYWxlKSJ9LHsidGV4dCI6Im91dHB1dDogIn1dfV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MiwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "parts": [
            {
                "text": "Write a character design in a pirate-themed game set in the Joseon era."
            },
            {
                "text": "input: Main Character (Female)"
            },
            {
                "text": "output: **Name:** Haenyeo\\n\\n**Appearance:**\\n\\n* **Age:** Mid-twenties\\n* **Height:** 5'7\\\"\\n* **Weight:** 130 lbs\\n* **Build:** Athletic and agile\\n* **Hair:** Long, black hair worn in a practical braid\\n* **Eyes:** Piercing hazel eyes that reflect the sea's depths\\n* **Clothing:**\\n    * Loose, dark-colored pants tucked into sturdy leather boots\\n    * A lightweight cotton shirt with a low neckline, trimmed with intricate embroidery\\n    * A leather vest with multiple pockets for tools and supplies\\n* **Accessories:**\\n    * A broad-brimmed hat to shield her face from the sun\\n    * A pair of silver earrings in the shape of anchors\\n    * A leather belt with a small pouch for valuables\\n\\

## Call `generate_content`

In [3]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

**Name:** Dohyeong

**Appearance:**

* **Age:** Late twenties
* **Height:** 6'0"
* **Weight:** 170 lbs
* **Build:** Lean and wiry, built for speed and agility.
* **Hair:** Dark brown, pulled back into a topknot, with a few loose strands framing his face.  
* **Eyes:** Sharp, dark brown eyes that often hold a mischievous glint.
* **Clothing:**
    * Loose-fitting baji pants (traditional Korean trousers) in a dark indigo color.
    * A short-sleeved, linen jeogori (traditional Korean jacket) in a lighter shade of blue, often left open to reveal a toned chest.  A dark-colored vest might be worn over this in colder weather.
    * Simple straw sandals or leather boots depending on the situation.
* **Accessories:**
    * A dojang (Korean headwear often associated with scholars), though his would be slightly more weathered and worn. It can be adjusted to shadow his face.
    * A jangdo (Korean sword) strapped to his hip.
    * A small, intricately carved wooden pendant hanging from a leather cord around his neck.  This might be a family heirloom or a memento of a lost love.


**Personality:**

* **Cunning:** Dohyeong is a master strategist and manipulator, always thinking several steps ahead.
* **Charismatic:** He possesses a natural charm and a quick wit, able to talk his way into (or out of) almost any situation.
* **Ambitious:**  Dohyeong craves power and recognition, dreaming of one day commanding his own fleet.
* **Loyal (to a select few):** While he can be deceitful to his enemies, he shows fierce loyalty to the few people he truly trusts.
* **Reckless:** His ambition sometimes leads him to take unnecessary risks.


**Skills:**

* **Swordsmanship:** Dohyeong is a highly skilled swordsman, specializing in quick, precise strikes.
* **Navigation:** He possesses an innate understanding of the sea and the stars, allowing him to navigate even the most treacherous waters.
* **Leadership:** Dohyeong has a natural talent for inspiring and motivating others, though his methods can sometimes be questionable.
* **Multilingual:** He is fluent in several languages, including Korean, Japanese, and Mandarin, which proves useful in his dealings with other pirates and traders.
* **Gambling:**  Dohyeong is a skilled gambler, often using his cunning and charisma to win big at cards or dice.


**Background:**

Dohyeong was born into a noble family, but his rebellious nature and thirst for adventure led him to abandon his privileged life and embrace the pirate life. His sharp intellect and strategic mind quickly earned him a prominent position within a crew, and he continues to rise through the ranks, leaving a trail of outwitted rivals in his wake. He carries a deep-seated resentment against the corrupt officials who he believes wronged his family, fueling his ambition for power and revenge.


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [4]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Write a character design in a pirate-themed game set in the Joseon era. 

input: Main Character (Female) 

output: **Name:** Haenyeo\n\n**Appearance:**\n\n* **Age:** Mid-twenties\n* **Height:** 5'7\"\n* **Weight:** 130 lbs\n* **Build:** Athletic and agile\n* **Hair:** Long, black hair worn in a practical braid\n* **Eyes:** Piercing hazel eyes that reflect the sea's depths\n* **Clothing:**\n    * Loose, dark-colored pants tucked into sturdy leather boots\n    * A lightweight cotton shirt with a low neckline, trimmed with intricate embroidery\n    * A leather vest with multiple pockets for tools and supplies\n* **Accessories:**\n    * A broad-brimmed hat to shield her face from the sun\n    * A pair of silver earrings in the shape of anchors\n    * A leather belt with a small pouch for valuables\n\n**Personality:**\n\n* **Determined:** Haenyeo is a fearless and tenacious pirate, driven by a strong sense of justice and a desire for adventure.\n* **Resourceful:** Her time spent as a haenye